## Генерирование текста рекуррентными сетями

Рассмотрим задачу генерации текста с помощью рекуррентой нейронной сети. Есть два варианта генерации:

* **Посимвольно**. В этом случае сеть как бы заново учится читать, при этом возможно создание некоторых новых интересных слов, однако на воспроизведение осмысленного текста требует больше обучающих ресурсов. 
* **По словам** - в этом случае сеть использует имеющиеся в тексте слова, и учится складывать их вместе.

Выбираем используемый способ обучения:

In [55]:
char_level = True # посимвольно
lower = False # приводить ли всё к нижнему регистру
max_words = 10000 # макс.число слов, которое учитывать в тексте (если char_level=False)

### Подготовка данных

Далее необходимо загрузить текст, на котором мы будем обучаться. Мы будем использовать текст старых ФИДОнет-конференций.

In [35]:
!wget http://www.soshnikov.com/temp/data/fido7.ru.anekdot.sample.gz
!gzip -d fido7.ru.anekdot.sample.gz

--2023-03-19 21:16:10--  http://www.soshnikov.com/temp/data/fido7.ru.anekdot.sample.gz
Resolving www.soshnikov.com (www.soshnikov.com)... 79.137.227.122
Connecting to www.soshnikov.com (www.soshnikov.com)|79.137.227.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70708194 (67M) [message/news]
Saving to: ‘fido7.ru.anekdot.sample.gz.1’

        fido7.ru.an   0%[                    ]  44.87K  82.8KB/s               ^C
gzip: fido7.ru.anekdot.sample already exists; do you wish to overwrite (y or n)? ^C


Загружаем текст из файла в переменную. Мы явно указываем кодировку текста - КОИ-8. Если вы загрузили хороший текст в Unicode - кодировку лучше не указывать.

In [56]:
text = [x.strip() for x in open('fido7.ru.anekdot.sample','r',encoding='koi8-r').readlines() 
         if not x.startswith('X-FTN')]
text[:15]

['From nobody Fri Sep  5 01:16:19 2014',
 'Path: news5.aus1.giganews.com!firehose2!nntp4!intern1.nntp.aus1.giganews.com!border1.nntp.aus1.giganews.com!nntp.giganews.com!newsfeed.gamma.ru!Gamma.RU!ddt.demos.su!f400.n5020!f4441.n5020!f52.n5020!f5.n5022!f43.n5022!f59.n5022!f57.n5022!not-for-mail',
 'Newsgroups: fido7.ru.anekdot',
 'Distribution: fido7',
 'X-Comment-To: All',
 'Approved: gateway@fido7.ru',
 'From: Andrey Chekalin <Andrey.Chekalin@p20.f57.n5022.z2.fidonet.org>',
 'Date: Mon, 07 Jul 2003 23:13:14 +0400',
 'Subject: test',
 'Message-ID: <1057623203@p20.f57.n5022.z2.ftn>',
 'Organization: -= Alien Station =-',
 '400/462',
 '464/34',
 '5005/14',
 '5012/8']

### Преобразование текста в числа

In [57]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

tokenizer = keras.preprocessing.text.Tokenizer(char_level=char_level,lower=lower,num_words=max_words)
tokenizer.fit_on_texts(['\n'.join(text[:100000])])
vocab_size = min(len(tokenizer.word_index),max_words)
print(f"Vocab size={vocab_size}")
tokenizer.texts_to_sequences(['From: Вася Пупкин'])

Vocab size=200


[[71, 13, 7, 29, 22, 1, 95, 19, 36, 55, 1, 93, 49, 47, 41, 31, 33]]

In [58]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ('' if char_level else ' ').join([reverse_map[t] for t in x])

Создаём из текста датасет, на котором потом сможем обучать нейросеть.

In [59]:
sequence_size = 1024 if char_level else 128 

def seq_generator(x,sequence_size=sequence_size,buffer_size=10000):
  for sub in range(0,len(x),buffer_size):
    t = tokenizer.texts_to_sequences(['\n'.join(x[sub:sub+buffer_size])])[0]
    for i in range(len(t)-sequence_size-1):
      yield t[i:i+sequence_size], t[i+1:i+1+sequence_size]

ds = tf.data.Dataset.from_generator(lambda : seq_generator(text),output_types=(tf.int32,tf.int32))

for x,y in ds.batch(32):
  print(x.shape,y.shape)
  break



(32, 1024) (32, 1024)


Определяем архитектуру нейросети. Можно поэкспериментировать с 

In [60]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size,50),
    #keras.layers.Bidirectional(keras.layers.LSTM(128,return_sequences=True)),
    #keras.layers.Bidirectional(keras.layers.LSTM(128,return_sequences=True)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 50)          10000     
                                                                 
 lstm_7 (LSTM)               (None, None, 128)         91648     
                                                                 
 dense_4 (Dense)             (None, None, 200)         25800     
                                                                 
Total params: 127,448
Trainable params: 127,448
Non-trainable params: 0
_________________________________________________________________


In [61]:
def generate(model,size=100,start='Lines: ',temperature=1.0,memory_size=128):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(inp,0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            chars.append(nc)
            inp = inp+[nc]
            inp = inp[:memory_size]
        return decode(chars)

generate(model)

'Lines: +/чOВ┐uИ©Y5у■RЩt|МTЬuB1ЗwmcnoФ┘O/дъЮ-8ПmЯN$l╕[ВПьЯE>N©@И4$шNAd}┐1╦╘r1гС▀]Хш░Ш$ 6t╩Qg\nf*gюb_3Ф@┐bmЯ\\c'

In [62]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model,size=500))
)

model.fit(ds.batch(32),callbacks=[sampling_callback],steps_per_epoch=3000,epochs=10)

Epoch 1/10
3000/3000 [==============================] - ETA: 0s - loss: 1.7148Lines: 7 00 Jull
"Hа бакнивы,  да, пpи экобёй Внейть ичкод оpжво-
бом пpочеpя в зажиз колазе конкоть вогоp?
"ОГЛАА шив ЦЯ "ай с вствиpя сомлючед ямpyшитy пpодвлем нза оpдаё
неpеснию пошyте, оли с бкy до насти"
"Тыволюсьстиникy. Вы (что чего собкой помда!
"Р
Кочниз yктпаp!"И нiташиве нивиpом од желивиствёpь y и пpоевлескаp поpоp выи
Изатвеpная одцаpый всостю к y вкого кобЭо?
5
47.
"Дось жака)И госто-
остyкые ."Hе yт вы бpоеа и заpавить из в тосбел
"Эаенивый из пpимотвеннy-
"Рyб,,
ко-
""ЯАЯ Г,  пыво-
мyд
3000/3000 [==============================] - 151s 49ms/step - loss: 1.7148
Epoch 2/10
2999/3000 [============================>.] - ETA: 0s - loss: 1.0159Lines: f44!news.com!prgacov.net.figanews.com!border1.nntp.aus1.giganews.com!coh!ndt!news.amma.ru!newsfeed.gamma.ru!Gamma.RU!ddt.demos.su!f400.n50!n50!34!f758.n5020!f20.n460!f20.n50!fidorg>
Dalizatiov <dred: Dmatr: Skankion: 0003.13 201.z2032.z2.fidon25!intn>
OT

In [ ]:
def print_output(x):
  for t in x.split('\n'):
    print(t)

for temp in [0.8,0.9,1]:
  print(f'=== temperature: {temp} ===')
  print_output(generate(model,start='Lines:',size=6000,temperature=temp))

=== temperature: 0.8 ===
Lines:7.u4!ron-xtidot!easynews.com!easynews.com!easynews!news.runnet.ru!news-zero.demos.su!ddt.demos.su!f400.n5020!f4401.n5020!f52.n5020!f174.n469!f52.n5020!f930.nus1.giganews.com!border1.nntp.aus1.giganews.com!nntp.giganews.com!newsfeed.runnet.ru!news-zero.demos.su!ddt.demos.su!f400.n5020!f4441.n5020!f52.n5020!f4441.n5020!f52.n5020!f220.n463!f4.n4504.z2.frn>
Organizator.of.RU.ANEKDOT@f32.n468.z2.fidonet.org>
Date: Sun, 10 Aug 2003 11:44:05 +0400
Subject: -Даниконфаpпами нам наказывается анекомиливаятоговствитсяк дессить получать как сонфем.
- Мопимеником,  не дако полмечается пpавлиние попpетела о накают
pyчикие бъеpя
допельмие pе-таникноговильется песлазилсение пистоо какиз всегок Попистопи дне чеpом пpавились пись бyдоние писекхи, копомить копpашовие, конфегpотек.

F>                                                                                      нас напигиваться то бyдетается, пpавил не кодпим - lix углаегли на учто назамается то пpавилимы имочлет
[ Bo